In [1]:
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet
!pip install numpy --quiet
!pip install geopy --quiet

In [2]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as pt
from geopy.geocoders import Nominatim
import re

pt.style.use('Solarize_Light2')

%matplotlib inline

In [3]:
df1 = pd.read_csv('FB_AutomotiveRepair.csv')
df2 = pd.read_csv('FB_AutomotiveService.csv')
df3 = pd.read_csv('FB_AutoService.csv')
df4 = pd.read_csv('FB_CarRepair.csv')
df5 = pd.read_csv('FB_Garage.csv')

df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

df.head()

,page_name,page_category,email,page_website,social_media_links,phone_number,location,page_rate,page_review_number,page_likes,page_followers,following,url
0,Automotive Repair,Automotive Repair Shop,automotiverepair@gmail.com,http://Automotiverepair.com,NaN,+1 202-555-1512,"400 5th Ave, Little Italy, Manhattan, New York...",NaN,NaN,157,12K,NaN,https://www.facebook.com/profile.php?id=615506...
1,Automotive Repair,Automotive Repair Shop,automotiverepair6@gmail.com,https://goo.gl/maps/dvBJwQqLr4U9JTYF8,NaN,+503 7454 4785,"https://goo.gl/maps/dvBJwQqLr4U9JTYF8, San Sal...",NaN,NaN,1.6K,2K,NaN,https://www.facebook.com/profile.php?id=100090...
2,Drick Shop Automotive Repair &amp; Installation,Automotive Repair Shop,NaN,NaN,NaN,NaN,"Paranaque city, Parañaque, Philippines, 1719",NaN,NaN,254,335,NaN,https://www.facebook.com/Drickshop.automotive.ph
3,D'CARs Automotive Repair SHOP,Automotive Repair Shop,dcarsautomotiverepairshop@gmail.com,NaN,NaN,NaN,"134 Mindanao ave, Quezon City, Philippines, 1101",NaN,NaN,147,522,NaN,https://www.facebook.com/profile.php?id=615541...
4,Ex Manila Automotive Repair Shop,Automotive Customization Shop,NaN,NaN,NaN,NaN,"1311 Pres Qurino Ave Paco , Manila, Philippine...",100.0,14.0,3.9K,4.2K,NaN,https://www.facebook.com/exmanila1


In [4]:
df.isna().sum()

page_name                0
page_category            0
email                 1964
page_website          3170
social_media_links    4848
phone_number          2736
location               509
page_rate             3904
page_review_number    3904
page_likes             367
page_followers           0
following             4707
url                      0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5074 entries, 0 to 5073
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   page_name           5074 non-null   object 
 1   page_category       5074 non-null   object 
 2   email               3110 non-null   object 
 3   page_website        1904 non-null   object 
 4   social_media_links  226 non-null    object 
 5   phone_number        2338 non-null   object 
 6   location            4565 non-null   object 
 7   page_rate           1170 non-null   float64
 8   page_review_number  1170 non-null   float64
 9   page_likes          4707 non-null   object 
 10  page_followers      5074 non-null   object 
 11  following           367 non-null    object 
 12  url                 5074 non-null   object 
dtypes: float64(2), object(11)
memory usage: 515.5+ KB


In [6]:
df.describe()

,page_rate,page_review_number
count,1170.000000,1170.000000
mean,95.900855,21.662393
std,6.871452,55.854844
min,42.000000,5.000000
25%,94.000000,7.000000
50%,100.000000,11.000000
75%,100.000000,22.000000
max,100.000000,975.000000


In [7]:
df = df.drop_duplicates(subset=['page_name', 'page_category'])

df.shape

(4767, 13)

In [8]:
df = df.drop(columns=['following'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4767 entries, 0 to 5073
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   page_name           4767 non-null   object 
 1   page_category       4767 non-null   object 
 2   email               2903 non-null   object 
 3   page_website        1768 non-null   object 
 4   social_media_links  213 non-null    object 
 5   phone_number        2144 non-null   object 
 6   location            4291 non-null   object 
 7   page_rate           1065 non-null   float64
 8   page_review_number  1065 non-null   float64
 9   page_likes          4424 non-null   object 
 10  page_followers      4767 non-null   object 
 11  url                 4767 non-null   object 
dtypes: float64(2), object(10)
memory usage: 484.1+ KB


In [9]:
df = df.dropna(subset=['location'])

df.shape

(4291, 12)

In [10]:
df['page_category'].unique()

filtered_categories = ['Automotive Repair Shop', 'Automotive Customization Shop',
       'Automotive Parts Store', 'Automotive Service',
       'Automotive, Aircraft & Boat', 'Local service',
       'Automotive Repair Shop · Truck Repair Shop · Motorcycle Repair Shop',
       'Product/service', 'Motor vehicle company', 
       'Tools/Equipment', 'Automotive Service · Automotive Repair Shop',
       'Automotive Repair Shop · Automotive Service',
       'Automotive Repair Shop · Emergency Roadside Service · Automotive Service',
       'Oil Lube & Filter Service',
       'Automotive Service · Product/service · E-commerce website',
       'Local service · Automotive Repair Shop', 'Automotive Body Shop',
       'Automotive Repair Shop · Tire Dealer & Repair Shop · Oil Lube & Filter Service',
       'Automotive Repair Shop · Tire Dealer & Repair Shop · Local service',
       'Tire Dealer & Repair Shop',
       'Automotive Restoration Service',
       'Business service',
       'Auto Detailing Service', 'Automotive Consultant', 'Cars',
       'Automotive Repair Shop · Tire Dealer & Repair Shop · Automotive Service',
       'Automotive Store', 'Engineering Service',
       'Automotive Repair Shop · Automotive Service · Oil Lube & Filter Service',
       'Car dealership', 'Automotive Repair Shop · Truck Repair Shop',
       'Automotive Repair Shop · Heating, Ventilating & Air Conditioning Service · Oil Lube & Filter Service',
       'Local business',
       'Automotive Service · Automotive Repair Shop · Car dealership',
       'Automotive Service · Diagnostic Center', 'Truck Repair Shop',
       'Motorcycle Repair Shop',
       'Towing Service · Automotive Service · Automotive Repair Shop',
       'Automotive Repair Shop · Automotive Service · Automotive, Aircraft & Boat',
       'Automotive Repair Shop · Oil Lube & Filter Service · Automotive Service',
       'Automotive Repair Shop · Automotive, Aircraft & Boat',
       'Emergency Roadside Service · Automotive Repair Shop · Automotive Service',
       'Automotive Customization Shop · Automotive Repair Shop · Oil Lube & Filter Service',
       'Oil Lube & Filter Service · Automotive Repair Shop · Automotive Service',
       'Automotive Store · Car dealership',
       'Automotive Repair Shop · Automotive Parts Store · Motorcycle Repair Shop',
       'Tire Dealer & Repair Shop · Oil Lube & Filter Service · Automotive Repair Shop',
       'Product/service · Automotive, Aircraft & Boat',
       'Automotive Repair Shop · Automotive Body Shop',
       'Oil Lube & Filter Service · Tire Dealer & Repair Shop · Automotive Service',
       'Automotive Body Shop · Automotive Repair Shop · Towing Service',
       'Automotive Repair Shop · In-Home Service',
       'Car Wash · Automotive Service',
       'Automotive Wholesaler',
       'Automotive Service · Automotive Parts Store',
       'Automotive Dealership',
       'Agricultural Service · Automotive Service',
       'Automotive Service · Car Wash',
       'Automotive Parts Store · Hardware Store',
       'Automotive Repair Shop · Truck Repair Shop · Car dealership',
       'Automotive Repair Shop · Automation Service · Automotive, Aircraft & Boat',
       'Automation Service · Automotive Repair Shop',
       'Automotive Repair Shop · Truck Repair Shop · Oil Lube & Filter Service',
       'Automotive Repair Shop · Oil Lube & Filter Service · Tire Dealer & Repair Shop',
       'Car dealership · Automotive, Aircraft & Boat',
       'Automotive Repair Shop · Automotive Service · Car dealership',
       'Car dealership · Commercial Truck Dealership · Commercial & Industrial Equipment Supplier',
       'Automotive Service · Locksmith · Car dealership',
       'Used Vehicles · Automotive Repair Shop · Cars',
       'Tire Dealer & Repair Shop · Automotive Repair Shop',
       'Professional Service', 'Automotive Manufacturer',
       'Automotive Repair Shop · Truck Repair Shop · Automotive Service', 'In-Home Service',
       'Automotive Repair Shop · Automation Service',
       'Auto Detailing Service · Car Wash · Automotive Customization Shop',
       'Automotive Repair Shop · Tire Dealer & Repair Shop · Automotive Restoration Service',
       'Automotive Restoration Service · Automotive Repair Shop · Tire Dealer & Repair Shop',
       'Tire Dealer & Repair Shop · Automotive Restoration Service · Automotive Repair Shop',
       'Automotive Repair Shop · Automotive Body Shop · Smog Emissions Check Station', 'Cars · Automotive Parts Store',
       'Automotive Repair Shop · Automotive Parts Store · Tire Dealer & Repair Shop',
       'Car Rental', 'Wheel & Rim Repair Service',
       'Automotive Repair Shop · Automotive Parts Store',
       'Car Wash · Auto Detailing Service',
       'RV Repair Shop',
       'Automotive Customization Shop · Automotive Parts Store · Auto Detailing Service',
       'Automotive Repair Shop · Automotive Body Shop · Automotive Parts Store',
       'Automotive Repair Shop · RV Repair Shop · Tire Dealer & Repair Shop',
       'Automotive Repair Shop · Auto Detailing Service · Car Wash',
       'Cars · Automotive Service · Automotive Repair Shop',
       'Automotive Repair Shop · Car Wash',
       'Heating, Ventilating & Air Conditioning Service',
       'Automotive Repair Shop · Automotive Body Shop · Tire Dealer & Repair Shop',
       'Tire Dealer & Repair Shop · Automotive Repair Shop · Auto Detailing Service',
       'Car Wash · Auto Detailing Service · Automotive Restoration Service',
       'Automotive Repair Shop · Cars · Car dealership',
       'Automotive Window Tinting Service',
       'Automotive Leasing Service',
       'Automotive Repair Shop · Automotive Glass Service · Automotive Parts Store',
       'Automotive Repair Shop · Automotive Parts Store · Automotive Glass Service',
       'Automotive Repair Shop · Car Wash · Automotive Service',
       'Auto Detailing Service · Automotive Repair Shop · Automotive Body Shop',
       'Automotive Service · Towing Service · Automotive Repair Shop',
       'Automotive Repair Shop · Car Rental · Automotive Service',
       'Automotive Repair Shop · Automotive Dealership',
       'Automotive Body Shop · Electronics',
       'Automotive, Aircraft & Boat · Local service', 'Automotive Body Shop · Automotive Repair Shop',
       'Auto Detailing Service · Automotive Repair Shop · Automotive Dealership',
       'Automotive Service · Automotive Body Shop · RV Repair Shop',
       'Automotive Repair Shop · Oil Lube & Filter Service',
       'Automotive Repair Shop · Automotive Body Shop · Auto Detailing Service',
       'Automotive Repair Shop · Towing Service',
       'Automotive Repair Shop · Automotive Service · Auto Detailing Service',
       'Cars · Local service · Automotive, Aircraft & Boat',
       'Automotive Repair Shop · Automotive Service · In-Home Service',
       'Automotive Repair Shop · Automotive Service · Automotive Body Shop',
       'Accessories · Business Center · Auto Detailing Service',
       'Car dealership · Automotive Repair Shop',
       'Automotive Repair Shop · Used Vehicles · Automotive Service',
       'Heating, Ventilating & Air Conditioning Service · Automotive Repair Shop',
       'Automotive Parts Store · Automotive Body Shop',
       'Automotive Repair Shop · Wheel & Rim Repair Service · Tire Dealer & Repair Shop',
       'Automotive, Aircraft & Boat · Musician/band · Heating, Ventilating & Air Conditioning Service',
       'Automotive, Aircraft & Boat · Automotive Repair Shop · Car Wash',
       'Automotive Body Shop · Automotive Repair Shop · Automotive Restoration Service',
       'Cars · Automotive Service · Automotive Parts Store',
       'Car Rental · Cars · Automotive Repair Shop',
       'Automotive Repair Shop · Automotive Parts Store · Shopping & retail',
       'Car Wash · Garage Door Service',
       'Motorcycle Dealership · Car dealership · Motorcycle Repair Shop',
       'Motorcycle Repair Shop · Motorcycle Dealership · Tire Dealer & Repair Shop',
       'Motorcycle Dealership · Motorcycle Repair Shop · Motorcycle Manufacturer',
       'Car dealership · Motorcycle Dealership · Auto Detailing Service', 'Automotive Parts Store · Motorsports Store',
       'Car dealership · Motorcycle Dealership',
       'Auto Detailing Service · Automotive Window Tinting Service · Automotive Service',
       'Automotive Service · Accessories · Automotive Parts Store', 'Car dealership · Accessories',
       'Motorcycle Repair Shop · Automotive Repair Shop · Motor vehicle company',
       'Automotive Repair Shop · Car dealership',
       'Car dealership · Cars',
       'Auto Detailing Service · Motorcycle Dealership',
       'Car Wash · Auto Detailing Service · Automotive, Aircraft & Boat',
       'Motorsports Store · Motorcycle Repair Shop · Automotive Parts Store',
       'Automotive Window Tinting Service · Auto Detailing Service · Automotive Service',
       'Motorcycle Repair Shop · Motorsports Store',
       'Motorcycle Dealership · Motor vehicle company',
       'Motorcycle Dealership · Motor vehicle company · Motorsports Store',
       'Motorsports Store · Sporting Goods Store',
       'Automotive Window Tinting Service · Auto Detailing Service · Cars', 'Motorcycle Dealership · Car dealership',
       'Auto Detailing Service · Automotive Restoration Service · Automotive Window Tinting Service',
       'Commercial Vehicle Dealership',
       'Automotive, Aircraft & Boat · Shopping & retail',
       'Used Vehicles · Car dealership',
       'Automotive Parts Store · Motorsports Store · Automotive Service',
       'Automotive Repair Shop · Motorcycle Repair Shop · Shopping & retail',
       'Motorcycle Dealership · Motorsports Store · Car dealership',
       'Cars · Car dealership', 'Computer Repair Service',
       'Accessories · Automotive, Aircraft & Boat',
       'Cars · Automotive Service · Car dealership',
       'Motorcycle Repair Shop · Wholesale & Supply Store',
       'Motor vehicle company · Motorcycle Dealership · Car dealership', 
       'Automotive Repair Shop · Tire Dealer & Repair Shop',
       'Automotive Parts Store · Motorcycle Repair Shop · Accessories',
       'Car dealership · Motorcycle Manufacturer · Motor vehicle company']

df = df[df['page_category'].isin(filtered_categories)]

df.shape

(3564, 12)

In [11]:
locations = ['Manila', 'Batangas', 'Laguna', 'Cavite', 'Rizal', 'Bulacan', 'Mandaluyong', 'Marikina', 'Pasig', 'Quezon City', 'San Juan', 'Caloocan', 'Malabon', 'Navotas', 'Valenzuela', 'Makati', 'Muntinlupa', 'Las Piñas', 'Parañaque', 'Pasay', 'Pateros', 'Taguig']

location_pattern = '|'.join([re.escape(loc) for loc in locations])

df = df[~df['location'].str.contains('Not yet rated', na=False)]
df = df[df['location'].str.contains('Philippines', na=False)]
df = df[df['location'].str.contains(location_pattern, na=False, case=False)]

df.shape

(567, 12)

In [12]:
remove = ['Rapide', 'Mechanigo', 'Rapidé']

remove_pattern = '|'.join([re.escape(rm) for rm in remove])

df = df[~df['page_name'].str.contains(remove_pattern, na=False, case=False)]

df.shape

(530, 12)

In [13]:
def convert(value):
    if pd.isna(value):
        return None
    
    if isinstance(value, str): 
        value = value.strip()
        value = value.replace('follower', '') 
        
        if 'K' in value:
            return int(float(value.replace('K', '')) * 1000)
        
        if '.' in value:
            return int(float(value) * 1000)
        
        return int(value)
    
    return int(value)

df['page_followers'] = df['page_followers'].apply(convert)
df['page_likes'] = df['page_likes'].apply(convert)
    
df = df.sort_values(by=['page_followers', 'page_likes', 'page_rate', 'page_review_number'], ascending=[False, False, False, False])

In [14]:
df = df.reindex(columns=['page_name', 'page_category', 'location', 'page_followers', 'page_likes', 'page_rate', 'page_review_number', 'url', 'email', 'page_website', 'social_media_links', 'phone_number'])

In [15]:
df.to_csv('sorted_data.csv', index=False)